In [ ]:
#pip install scikit-optimize
!pip install -q -U keras-tuner
# !pip install hyperopt


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import math
import numpy as np
import tensorflow as tf
import keras_tuner as kt
import tensorflow_datasets as tfds
from tensorflow import keras
from tensorflow.keras import layers
from sklearn import preprocessing
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.utils.random import sample_without_replacement
from sklearn.utils import resample
from tensorflow.tools.docs import doc_controls


In [ ]:
train_dataset = np.load('train_dataset_1k_60.npz')
val_dataset = np.load('validation_dataset_1k_40.npz')
test_dataset = np.load('test_dataset.npz')

In [ ]:
# Preprocess the data (these are NumPy arrays)
x_train = train_dataset['x'].reshape(600, 784).astype("float32") / 255
x_test = test_dataset['x'].reshape(10010, 784).astype("float32") / 255
y_train = train_dataset['y'].astype("float32")
y_test = test_dataset['y'].astype("float32")
x_val = val_dataset['x'].reshape(400, 784).astype("float32") / 255
y_val = val_dataset['y'].astype("float32")

In [ ]:
# lamdas = kt.Float("l2", min_value=1e-4, max_value=1e-2, sampling="log")
# print(lamdas)
# hp = kt.HyperParameters()
# print(hp.Int("units", min_value=32, max_value=512, step=32))


In [ ]:
class CustomLoss(keras.losses.Loss):
    def __init__(self,model, lamda1, lamda2, name="custom_loss"):
        super().__init__(name=name)
        # print(lamda)
        self.regularization_factor1 = tf.math.exp(lamda1)
        self.regularization_factor2 = tf.math.exp(lamda2)
        self.w1 = model.layers[1].weights[0]
        self.w2 = model.layers[2].weights[0]


    def call(self, y_true, y_pred):
        scce = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
        loss = scce(y_true, y_pred)
        reg = self.regularization_factor1*tf.nn.l2_loss(self.w1) + self.regularization_factor2*tf.nn.l2_loss(self.w2)
        return loss + reg

In [ ]:
def build_model(hp):
  # temp
  lamda1 = hp.Float("lamda1", min_value=-10, max_value=-0.1,step=0.01)
  lamda2 = hp.Float("lamda2", min_value=-10, max_value=-0.1,step=0.01)
  model = tf.keras.Sequential([
      tf.keras.layers.Flatten(input_shape=(784,)),
      tf.keras.layers.Dense(100 ,activation='relu'),#,kernel_regularizer='l2'),
      tf.keras.layers.Dense(10)#,kernel_regularizer='l2')#,kernel_regularizer=keras.regularizers.l2())    # didn't use softmax since it will be called when (logits=true) in below step
  ])
  l_rate = hp.Float("l_rate", min_value=0.1,max_value=0.9,step = 0.01)
  momentum = hp.Float("momentum", min_value=0.01,max_value=0.1,step = 0.01)
  optimizer = keras.optimizers.SGD(learning_rate=l_rate,momentum = momentum )
  lamda1 = tf.cast(lamda1,dtype='float32')
  lamda2 = tf.cast(lamda2,dtype='float32')
  model.compile(optimizer=optimizer, loss=CustomLoss(model,lamda1,lamda2),metrics=[tf.keras.metrics.SparseCategoricalCrossentropy(from_logits=True)])
  return model

In [ ]:
# print(lamda1)

In [ ]:
tuner = kt.Hyperband(build_model,objective=kt.Objective("val_sparse_categorical_crossentropy",direction="min"),max_epochs=100,hyperband_iterations=1,overwrite=True)
tuner.search(x_train, y_train, validation_data=(x_val, y_val))
tuner.search_space_summary()

Trial 254 Complete [00h 00m 21s]
val_sparse_categorical_crossentropy: 0.436399906873703

Best val_sparse_categorical_crossentropy So Far: 0.4235987961292267
Total elapsed time: 00h 10m 27s
INFO:tensorflow:Oracle triggered exit
Search space summary
Default search space size: 4
lamda1 (Float)
{'default': -10.0, 'conditions': [], 'min_value': -10.0, 'max_value': -0.1, 'step': 0.01, 'sampling': None}
lamda2 (Float)
{'default': -10.0, 'conditions': [], 'min_value': -10.0, 'max_value': -0.1, 'step': 0.01, 'sampling': None}
l_rate (Float)
{'default': 0.1, 'conditions': [], 'min_value': 0.1, 'max_value': 0.9, 'step': 0.01, 'sampling': None}
momentum (Float)
{'default': 0.01, 'conditions': [], 'min_value': 0.01, 'max_value': 0.1, 'step': 0.01, 'sampling': None}


In [ ]:
models = tuner.get_best_models(num_models=1)   #get the best model
best_model = models[0]
# print(best_model.layers[1].get_weights())

In [ ]:
best_hps = tuner.get_best_hyperparameters(1)
print('-----------------------optimal hyperparameters----------------------------')

print(f'Optimal l_rate = {best_hps[0].get("l_rate")}')
print(f'Optimal momentum = {best_hps[0].get("momentum")}')
print(f'Optimal lamda1 = {best_hps[0].get("lamda1")}')
print(f'Optimal lamda1 = {best_hps[0].get("lamda2")}')

-----------------------optimal hyperparameters----------------------------
Optimal l_rate = 0.5999999999999998
Optimal momentum = 0.06999999999999999
Optimal lamda1 = -5.3000000000001
Optimal lamda1 = -6.050000000000084


## LOSS

In [ ]:
def calculate_loss(X,Y,name):
  loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
  logits = best_model(X)
  loss = loss_fn(Y,logits)
  print(f'{name}_loss = {loss}')

In [ ]:
calculate_loss(x_val,y_val,"validation")
calculate_loss(x_train,y_train,"training")
calculate_loss(x_test,y_test,"test")


validation_loss = 0.4235988259315491
training_loss = 0.035607483237981796
test_loss = 0.40626686811447144
